##Desáfio Join. Vaga para Analista de Dados Jr (python)

In [0]:
#lista de colunas existentes no banco
colunas = ['customers','employees','offices','orderdetails','orders','payments','product_lines','products']

Visualizando e Extraindo dados das tabelas e salvando no formato parquet

In [0]:
#função criada para conectar ao banco de dados 
def conexao():
 for colunas_lista in colunas:   
      driver = "org.postgresql.Driver"
      url = "jdbc:postgresql://psql-mock-database-cloud.postgres.database.azure.com:5432/ecom1692848222792bypjwxeymatmczia"
      table = f'{colunas_lista}'
      user = "gkwimjzggfdatutzxsdmvjzq@psql-mock-database-cloud"
      password = "ntikuznbgqhgqqbsydxnycdu"
      mostrar_tabelas_extrair(colunas_lista, driver, url, table, user, password)

In [0]:
#Função criada visualizar as tabelas do banco de dados e para extrair as tabelas no formato parquet
def mostrar_tabelas_extrair(colunas_lista, driver, url, table, user, password,):
  global df 
  df = (
    spark.read.format("jdbc") \
      .option("driver", driver)
      .option("url", url)
      .option("dbtable", table)
      .option("user", user)
      .option("password", password)
      .load()
  )
  df.write.format("parquet").mode("overwrite").save(f'dbfs:/FileStore/shared_uploads/ninivehelen@gmail.com/{colunas_lista}.parquet')
  print(f'arquivo: {colunas_lista} salvo')
  #display(df)


In [0]:
#Main para fazer a chamada das funções
if __name__=="__main__":
    conexao()

arquivo: customers salvo
arquivo: employees salvo
arquivo: offices salvo
arquivo: orderdetails salvo
arquivo: orders salvo
arquivo: payments salvo
arquivo: product_lines salvo
arquivo: products salvo


##Etapa de trazer informações com pyspark

Etapa com spark para trazer informações como:
- Qual país possui a maior quantidade de itens cancelados?
- Qual o faturamento da linha de produto mais vendido, considere como os itens
Shipped, cujo o pedido foi realizado no ano de 2005?
- Nome, sobrenome e e-mail dos vendedores do Japão, o local-part do e-mail
deve estar mascarado.

In [0]:
#Capturando os arquivos necessário para as consultas sql
customers = spark.read.format("parquet").option("header", "true").load("dbfs:/FileStore/shared_uploads/ninivehelen@gmail.com/customers.parquet")
orders = spark.read.format("parquet").option("header", "true").load("dbfs:/FileStore/shared_uploads/ninivehelen@gmail.com/orders.parquet")
products = spark.read.format("parquet").option("header", "true").load("dbfs:/FileStore/shared_uploads/ninivehelen@gmail.com/products.parquet")
payments = spark.read.format("parquet").option("header", "true").load("dbfs:/FileStore/shared_uploads/ninivehelen@gmail.com/payments.parquet")
orderdetails = spark.read.format("parquet").option("header", "true").load("dbfs:/FileStore/shared_uploads/ninivehelen@gmail.com/orderdetails.parquet")
offices = spark.read.format("parquet").option("header", "true").load("dbfs:/FileStore/shared_uploads/ninivehelen@gmail.com/offices.parquet")
employees = spark.read.format("parquet").option("header", "true").load("dbfs:/FileStore/shared_uploads/ninivehelen@gmail.com/employees.parquet")

In [0]:
#Criando uma tempView para fazer as consultas sql 
customers.createOrReplaceTempView("customers")
orders.createOrReplaceTempView("orders")
products.createOrReplaceTempView("products")
payments.createOrReplaceTempView("payments")
orderdetails.createOrReplaceTempView("orderdetails")
offices.createOrReplaceTempView("offices")
employees.createOrReplaceTempView("employees")

Qual país possui a maior quantidade de itens cancelados?

In [0]:
#Sql criado com spark para trazer a resposta da 1 pergunta 
df_inner_join_consulta_1 = spark.sql("""
select customers.country,orders.status, count(orders.status) as total_cancelados
from orders INNER JOIN  customers On  
customers.customer_number = orders.customer_number 
where orders.status = 'Cancelled'
group by customers.country, orders.status
ORDER BY total_cancelados DESC
""")
#Salvar consulta em formato delta
df_inner_join_consulta_1.write.format("delta").mode("overwrite").save('dbfs:/FileStore/shared_uploads/ninivehelen@gmail.com/pais_quant_mais_intens_cancelados.delta')
print('arquivo: pais_quant_mais_intens_cancelados salvo')
display(df_inner_join_consulta_1)

arquivo: pais_quant_mais_intens_cancelados salvo


country,status,total_cancelados
New Zealand,Cancelled,2
Spain,Cancelled,1
Sweden,Cancelled,1
USA,Cancelled,1
UK,Cancelled,1


**Resposta: New Zealand, Cancelled 2**

Qual o faturamento da linha de produto mais vendido, considere como os itens Shipped, cujo o pedido foi realizado no ano de 2005?

In [0]:
#Sql criado com spark para trazer a resposta da 2 pergunta 
df_inner_join_consulta_2 = spark.sql("""
select products.product_line, sum(payments.amount)as total_faturamento, orders.status, count(orders.status) as total_entregue  
FROM payments 
inner join orders on payments.customer_number = orders.customer_number
inner join orderdetails on orderdetails.order_number = orders.order_number
inner join products on products.product_code = orderdetails.product_code
where orders.status = 'Shipped' and Extract(year from orders.order_date) = '2005'
group by products.product_line, orders.status
ORDER BY total_faturamento DESC
""")
#Salvar consulta em formato delta
df_inner_join_consulta_2.write.format("delta").mode("overwrite").save('dbfs:/FileStore/shared_uploads/ninivehelen@gmail.com/fatu_linha_prod_mais_vendido.delta')
print('arquivo: fatu_linha_prod_mais_vendido salvo')
display(df_inner_join_consulta_2)

arquivo: fatu_linha_prod_mais_vendido salvo


product_line,total_faturamento,status,total_entregue
Classic Cars,47574393.22,Shipped,937
Vintage Cars,20554555.43,Shipped,423
Trucks and Buses,19052696.92,Shipped,367
Motorcycles,9481582.65,Shipped,228
Trains,4500938.31,Shipped,87
Planes,3954799.97,Shipped,113
Ships,3573772.73,Shipped,85


**Resposta: Classic Cars, 47574393.22**

3. Nome, sobrenome e e-mail dos vendedores do Japão, o local-part do e-mail deve estar mascarado.

In [0]:
#Sql criado com spark para trazer a resposta da 3 pergunta 
df_inner_join_consulta_3 = spark.sql("""
select employees.first_name, employees.last_name,
regexp_replace(employees.email, '(.{2}(.@)+(classic)+)', '****') AS masked_email 
FROM employees 
inner join offices on employees.office_code = offices.office_code
where offices.country = 'Japan'
""")
df_inner_join_consulta_3.write.format("delta").mode("overwrite").save('dbfs:/FileStore/shared_uploads/ninivehelen@gmail.com/nome_sobrenome_vend_japao.delta')
print('arquivo: nome_sobrenome_vend_japao salvo')
display(df_inner_join_consulta_3)

arquivo: nome_sobrenome_vend_japao salvo


first_name,last_name,masked_email
Mami,Nishi,mni****modelcars.com
Yoshimi,Kato,yk****modelcars.com
